# Python 装饰器
  装饰器可以说是Python的一大特性，某些场合使用起来十分方便。但是很多人对装饰器的语法及使用十分困惑，本人也是经过反复查阅资料，反复学习才弄清搞懂。下面介绍一点学习的心得：
  
  - 应用场景
    
    装饰器是在不修改原函数API情况下，增加新的功能。举个最简单的例子，在函数执行前后打印一些信息。

In [45]:
def mydecorator(func):
    def inner(*args, **kwargs):
        print('before func execute')
        func(*args, **kwargs)
        print('after func execute')
    return inner

@mydecorator
def run(s):
    print(s)
    
run("test")

before func execute
test
after func execute


- 函数闭包
  
  装饰器本质上就是一个函数闭包。闭包简单一点说法就是指嵌套函数引用了外部函数的变量时，返回的不单单是内部函数本身，还包括所使用的外部函数变量的值。看下面的列子：

In [46]:
def mydecorator(func):
    name = 'zs'
    def inner(*args, **kwargs):
        print(name)
        func(*args, **kwargs)
    return inner

@mydecorator
def run():
    print("run")
    
run()

zs
run


 可以看到在经过装饰之后，mydecorator函数已经执行完毕， 但是调用run（）后，变量name的值仍然会被打印出来。装饰器return的并不仅仅是inner函数，还有它所使用的外部函数的变量name的值。
 
 - 语法糖@
 
 如果能准确把握@的语法，那么就可以轻松理解装饰器的内涵。简单来说可以将@看作是一种赋值运算符,下面通过一个列子来说明：

In [47]:
def mydecorator(func):
    def inner(*args, **kwargs):
        print('func id(before decorator):', id(func))
        func(*args, **kwargs)
    return inner

@mydecorator
def run(s):
    print(s)

run("XXXX")
print('func id(after decorator):',id(run))

func id(before decorator): 14458792
XXXX
func id(after decorator): 19885776


   对于@mydecorator一行， 可以认为在执行 run = mydecorator(run)。此时虽然run名字未变，但是所指向的函数已经发生改变。对于更加的复杂的装饰器，均可以通过这样解释语法糖符号@来理解。注意一点：对于装饰器来说，内层的函数必须和被装饰的函数具有相同的形式（参数、返回值）。程序中最终调用的函数（虽然名字未改变）就是装饰器的内层函数（通过上例的函数id可以看出来）。根据装饰器性质可知道，@后所跟的是一个装饰器，因此带参数的装饰器@decorator(args)，函数decorator(args)必须要返回一个装饰器:

In [48]:
def mydecorator(level):
    def wrapper(func):
        def inner(*args, **kwargs):
            print('level:', level)
            func(*args, **kwargs)
        return inner
    return wrapper

@mydecorator(level = 'Top')
def run(s):
    print(s)

run("XXXX")

level: Top
XXXX


上例，mydecorator返回的wrapper就是一个完整的装饰器。
- 类装饰器
以上的例子都是使用函数作为装饰器，用类作为装饰器，用法及解释方式也是一样的：

In [49]:
class mydecorator():
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print("before decorator")
        self.func(*args, **kwargs)
        print("after decorator")
        

@mydecorator
def run(s):
    print(s)
    
run('XXXXX')

before decorator
XXXXX
after decorator


按照函数装饰器的方式，我们来解释一下。当执行@mydecorator时， 相当于 run = mydecorator(func)， 此时run指向一个mydecorator对象。为了不改变函数的API，这个对象必须可以被调用（否则执行run("XXX")会报错），要在类里实现__call__方法，且方法的参数和返回值要跟被装饰函数保持一致。同理可以推出带参数的类装饰器的使用：

In [50]:
class mydecorator():
    def __init__(self, level):
        self.level = level
    
    def __call__(self, func):
        def inner(*args, **kwargs):
            print("level :", self.level)
            func(*args,**kwargs)
        return inner
        

@mydecorator(level='Top')
def run(s):
    print(s)
    
run('XXXXX')

level : Top
XXXXX


当执行当执行@mydecorator(level="Top")时， 相当于 run = mydecorator("Top")， run指向一个类对象，这个类对象必须是一个装饰器（即可以调用，参照第一个装饰器， func为参数，返回一个闭包），因此__call__方法的实现类似于第一个例子。
- 装饰器的执行及修饰顺序

当定义一个函数时，此函数并没有执行，只有在调用时代码才会被执行。

In [51]:
def run():
    print("run")

#run()

上述代码执行后，并不会被打印run，只有打开注释行才会打印。根据这一原则，推理出在执行语法糖@时，会执行相应的装饰器代码：

In [52]:
def mydecorator(func):
    print("decorator")
    def inner(*args, **kwargs):
        print('before func execute')
        func(*args, **kwargs)
        print('after func execute')
    return inner

@mydecorator
def run(s):
    print(s)
    
print("here")
run('XXX')

decorator
here
before func execute
XXX
after func execute


在代码执行print('here')之前，已经有输出decorator，装饰器函数的代码已经被执行。当有多个装饰器时，由于代码从上到下执行，当执行@A时，会把A之后包括@B的代码块当作一个被装饰函数（即执行A（一大块代码）），再解释“一大块代码”时发现@B，(可以认为先将A进栈)按照装饰器语法继续执行（B(run)），因此会先执行B(run)，然后再执行A(B(run)), 因此装饰器的代码会从下到上被执行，此时被装饰的函数并没有调用。而装饰的结果（调用被装饰的函数）却相反，从上到下（理解上可以把内层装饰看成一个整体before B、XXX、after B，一起被外层装饰）：

In [53]:
def A(func):
    print("A")
    def inner(*args, **kwargs):
        print('before A')
        func(*args, **kwargs)
        print('after A')
    return inner

def B(func):
    print("B")
    def inner(*args, **kwargs):
        print('before B')
        func(*args, **kwargs)
        print('after B')
    return inner

@A
@B
def run(s):
    print(s)
    
print("here")
run('XXX')

B
A
here
before A
before B
XXX
after B
after A


- 总的来说，理解装饰器牢记几点即可：
## 1. @是一个语法糖，相当于 func = decorator(func)；
## 2. 装饰器必须返回一个闭包函数，对类装饰器来讲类的对象必须可调用；
## 3. 返回的闭包函数必须跟被装饰的函数拥有相同的形式（参数，有无返回值等）。
## 4. 装饰器函数从下到上执行，从上到下装饰。